In [3]:
import os
from gensim.models.word2vec import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import multiprocessing
import gzip
import pandas as pd
import csv

In [4]:
tagged_titles = pd.read_csv('./dataset/Train_Tagged_Titles.tsv', sep='\t', on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf8')

In [5]:
tags = ['Accents', 'Brand', 'Character', 'Character Family', 'Closure', 'Color', 'Country/Region of Manufacture', 'Department', 'Fabric Type', 'Features', 'Handle Drop', 'Handle Style', 'Handle/Strap Material', 'Hardware Material', 'Lining Material', 'MPN', 'Material', 'Measurement, Dimension', 'Model', 'Occasion', 'Pattern', 'Pocket Type', 'Product Line', 'Season', 'Size', 'Strap Drop', 'Style', 'Theme', 'Trim Material', 'Type']

In [6]:
tiddies = tagged_titles.groupby('Record Number')['Token'].apply(list).to_dict()
ass = tagged_titles.groupby('Record Number')['Tag'].apply(list).to_dict()

kimk = [[(tiddies[i][tiddie], ass[i][tiddie]) for tiddie in range(0, len(ass[i]))] for i in range(1, len(ass) + 1)]

for i in range(0, len(kimk)):
    for j in reversed(range(1, len(kimk[i]))):
        if(kimk[i][j][1] != kimk[i][j][1]): #python nan moment
            kimk[i][j - 1] = (kimk[i][j - 1][0] + " " + kimk[i][j][0], kimk[i][j - 1][1])

delphine = [[i for i in item if not i[1] != i[1]] for item in kimk]

In [7]:
sentences = [[delphine[i][j][0] for j in range(len(delphine[i]))] for i in range(len(delphine))]

In [8]:
def create_wordvecs(corpus, model_name):
    from gensim.models.word2vec import Word2Vec
    from gensim.models.phrases import Phrases, Phraser
    from collections import defaultdict
    
    print (len(corpus))
    

    phrases = Phrases(corpus, min_count=30, progress_per=10000)
    print ("Made Phrases")
    
    bigram = Phraser(phrases)
    print ("Made Bigrams")
    
    sentences = phrases[corpus]
    print ("Found sentences")
    word_freq = defaultdict(int)

    for sent in sentences:
        for i in sent:
            word_freq[i]+=1

    print (len(word_freq))
    
    print ("Training model now...")
    w2v_model = Word2Vec(min_count=1,
                        window=2,
                        sample=6e-5,
                        alpha=0.03,
                        min_alpha=0.0007,
                        negative=20)
    w2v_model.build_vocab(sentences, progress_per=10000)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
    os.makedirs("./trainset", exist_ok=True)
    w2v_model.wv.save_word2vec_format(f"trainset/{model_name}.txt")
create_wordvecs(sentences, "word_vecs")

5000
Made Phrases
Made Bigrams
Found sentences
9505
Training model now...
